# Usando implementação BM25 do pyserini para executar as TREC-DL_2020 queries

Como a tarefa de **Passage Ranking** da trilha de Deep Learning da TREC 2020 utiliza o dataset **MS MARCO**, o [tutorial](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md) do "MS MARCO Passage Retrieval" descreve os passos principais que devem ser executados para a realização do item 1 do trabalho programado para a aula de 09/10/2023.

Este notebook captura todos os passos executados em uma máquina local para a execução dessa tarefa.

## Preparação do ambiente

Para a execução do [tutorial](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md) do "MS MARCO Passage Retrieval" é necessária a instalação do Pyserini e a sua dependência para buscas esparsas, o Anserini.

### Build do Anserini

Os passos indicados [aqui](https://github.com/castorini/anserini#-getting-started)para a instalação do Anserini funcionaram sem problemas.

Algo que não está claro, entretanto, é a necessidade de definição do CLASSPATH para o .jar do Anserini, uma vez que ele foi gerado:

    export ANSERINI_CLASSPATH={path-where-you-cloned}/anserini/target

### Instalação Pyserini

O [tutorial](https://github.com/castorini/pyserini/blob/master/docs/installation.md#development-installation) disponível para a instalação do Pyserini também foi suficiente para deixá-lo pronto para uso.

Os scripts executados neste notebook seguem o padrão do [tutorial](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md) do "MS MARCO Passage Retrieval", sendo executados a partir da pasta do **pysernini**.

In [1]:
import os

In [2]:
PYSERINI_FOLDER="/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini"

In [3]:
os.chdir(PYSERINI_FOLDER)

## Dowload do dataset MS MARCO

In [4]:
MSMARCO_PASSAGE_FOLDER="collections/msmarco-passage"

In [5]:
if os.path.exists(MSMARCO_PASSAGE_FOLDER):
    print("MSMARCO Passage dataset already downloaded...")
else:
    os.mkdirs(MSMARCO_PASSAGE_FOLDER)
    os.chdir(MSMARCO_PASSAGE_FOLDER)

    #### Dowload the MSMARCO dataset...
    
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
        
    !tar -xvfz collectionandqueries.tar.gz

    os.chdir(PYSERINI_FOLDER)

MSMARCO Passage dataset already downloaded...


## Conversão do dataset MS MARCO para o formato do pyserini

In [6]:
MSMARCO_DATASET_PYSERINI_FORMAT_FOLDER="collection_jsonl"

msmarco_dataset_path = os.path.join(MSMARCO_PASSAGE_FOLDER, "collection.tsv")
msmarco_pyserini_folder = os.path.join(MSMARCO_PASSAGE_FOLDER, MSMARCO_DATASET_PYSERINI_FORMAT_FOLDER)

In [7]:
if os.path.exists(msmarco_pyserini_folder):
    print("MSMARCO Passage dataset already converted to Pyserini's format...")
else:
    !python3.8 tools/scripts/msmarco/convert_collection_to_jsonl.py \
        --collection-path {msmarco_dataset_path} \
        --output-folder {msmarco_pyserini_folder}

MSMARCO Passage dataset already converted to Pyserini's format...


## Indexação do dataset

In [8]:
MSMARCO_LUCENE_INDEXES_FOLDER="indexes/lucene-index-msmarco-passage"

In [9]:
if os.path.exists(MSMARCO_LUCENE_INDEXES_FOLDER):
    print("MSMARCO Passage indexes already generated...")
else:
    !python3.8 -m pyserini.index.lucene \
        --collection JsonCollection \
        --input {msmarco_pyserini_folder} \
        --index {MSMARCO_LUCENE_INDEXES_FOLDER} \
        --generator DefaultLuceneDocumentGenerator \
        --threads 9 \
        --storePositions --storeDocvectors --storeRaw

MSMARCO Passage indexes already generated...


## Download das queries da tarefa de Passage Ranking do TREC-DL_2020

In [10]:
TRECDL2020_QUERIES_FOLDER="collections/trec-dl_2020-passage"

### Check if need to download the TREC-DL_2020 Passage Ranking task queries

In [11]:
if os.path.exists(TRECDL2020_QUERIES_FOLDER):
    print("TREC-DL_2020 queries already downloaded...")
else:
    os.mkdirs(TRECDL2020_QUERIES_FOLDER)
    os.chdir(TRECDL2020_QUERIES_FOLDER)

    #### Dowload the TREC-DL_2020 Passage Ranking task queries...

    !wget https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/qrels.dev.tsv
    !wget https://msmarco.blob.core.windows.net/msmarcoranking/qrels.train.tsv

    !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2020-queries.tsv.gz

    !tar -xvf queries.tar.gz

    !gunzip msmarco-test2020-queries.tsv.gz

    os.chdir(PYSERINI_FOLDER)

TREC-DL_2020 queries already downloaded...


## Execução das queries

In [12]:
from datetime import datetime

In [13]:
which_queries = "msmarco-test2020-queries.tsv"
execution_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

pyserini_runfile = "run.trec-dl_2020-passage.bm25tuned_{}_{}.txt".format(which_queries, execution_timestamp)
queries_path = os.path.join(TRECDL2020_QUERIES_FOLDER, which_queries)

In [14]:
!python3.8 -m pyserini.search.lucene \
    --index indexes/lucene-index-msmarco-passage \
    --topics {queries_path} \
    --output runs/{pyserini_runfile} \
    --output-format trec \
    --hits 1000 \
    --bm25 --k1 0.82 --b 0.68 \
    --threads 8

Setting BM25 parameters: k1=0.82, b=0.68
Running collections/trec-dl_2020-passage/msmarco-test2020-queries.tsv topics, saving to runs/run.trec-dl_2020-passage.bm25tuned_msmarco-test2020-queries.tsv_20230305_213031.txt...
100%|█████████████████████████████████████████| 200/200 [00:15<00:00, 12.80it/s]


## Avaliando o resultado

In [15]:
which_qrels = "2020qrels-pass.txt"
qrels_path = os.path.join(TRECDL2020_QUERIES_FOLDER, which_qrels)

In [16]:
!tools/eval/trec_eval.9.0.4/trec_eval -c -mrecall.1000 -mmap -mndcg_cut.10 -mrecip_rank \
    {qrels_path} runs/{pyserini_runfile}

map                   	all	0.3470
recip_rank            	all	0.8215
recall_1000           	all	0.7331
ndcg_cut_10           	all	0.4876
